In [225]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from train import load_dataset,train,eval
from sklearn.ensemble import StackingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedRandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [226]:
X_train, X_test, y_train, y_test = load_dataset(test_size=0.2)

In [227]:
(y_train==0).sum() / (y_train==1).sum()

np.float64(2.6991330953595103)

In [228]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=297) 

In [ ]:
estimators = [
    ('rf', make_pipeline(StandardScaler(),BalancedRandomForestClassifier(n_estimators=20,random_state=297))),
    ('lg', make_pipeline(StandardScaler(),
                          LogisticRegression(random_state=297,class_weight='balanced',max_iter=1000)))
]
clf = StackingClassifier(
    estimators=estimators, cv=cv,stack_method='predict_proba',n_jobs=4,final_estimator=CatBoostClassifier(n_estimators=1000,learning_rate=0.0010050000800100051124,depth=9,random_seed=297,auto_class_weights='Balanced',)
)

In [235]:
clf.fit(X_train,y_train)

0:	learn: 0.6900874	total: 11.4ms	remaining: 5.69s
1:	learn: 0.6871001	total: 19.8ms	remaining: 4.92s
2:	learn: 0.6843106	total: 29.1ms	remaining: 4.83s
3:	learn: 0.6813223	total: 36.3ms	remaining: 4.5s
4:	learn: 0.6786304	total: 45.2ms	remaining: 4.48s
5:	learn: 0.6755463	total: 53.9ms	remaining: 4.44s
6:	learn: 0.6730540	total: 62ms	remaining: 4.36s
7:	learn: 0.6703715	total: 70.5ms	remaining: 4.33s
8:	learn: 0.6676665	total: 79.2ms	remaining: 4.32s
9:	learn: 0.6649905	total: 88.9ms	remaining: 4.36s
10:	learn: 0.6626957	total: 96.6ms	remaining: 4.29s
11:	learn: 0.6599806	total: 104ms	remaining: 4.24s
12:	learn: 0.6571139	total: 112ms	remaining: 4.19s
13:	learn: 0.6541181	total: 119ms	remaining: 4.14s
14:	learn: 0.6514105	total: 128ms	remaining: 4.15s
15:	learn: 0.6494106	total: 138ms	remaining: 4.17s
16:	learn: 0.6469661	total: 146ms	remaining: 4.16s
17:	learn: 0.6446807	total: 154ms	remaining: 4.14s
18:	learn: 0.6421118	total: 163ms	remaining: 4.14s
19:	learn: 0.6397052	total: 172ms

,estimators,"[('rf', ...), ('rgb', ...)]"
,final_estimator,LogisticRegre...ax_iter=10000)
,cv,StratifiedKFo... shuffle=True)
,stack_method,'predict_proba'
,n_jobs,4
,passthrough,True
,verbose,0
,copy,True
,with_mean,True
,with_std,True
,n_estimators,20


In [236]:
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
print(classification_report(y_test,y_pred,labels=[0,1],digits=3,output_dict=False))
print(classification_report(y_train,y_pred_train,labels=[0,1],digits=3,output_dict=False))

              precision    recall  f1-score   support

           0      0.899     0.952     0.925      1323
           1      0.847     0.711     0.773       491

    accuracy                          0.887      1814
   macro avg      0.873     0.832     0.849      1814
weighted avg      0.885     0.887     0.884      1814

              precision    recall  f1-score   support

           0      0.922     0.978     0.949      5293
           1      0.929     0.776     0.846      1961

    accuracy                          0.923      7254
   macro avg      0.925     0.877     0.897      7254
weighted avg      0.924     0.923     0.921      7254



In [237]:
test_df = pd.read_csv("data/datatest.csv")
X = test_df.select_dtypes(include=["number"]).drop(columns=["Created At Year","Created At Month","Countries_ID"])

In [238]:
predictions = clf.predict(X)

In [239]:
predictions.sum()

np.int64(3)

In [224]:
sub = pd.DataFrame({
    'User_ID_Next_month_Activity': test_df.User_ID,
    'Active': predictions
})
sub.User_ID_Next_month_Activity = sub.User_ID_Next_month_Activity + '_Month_5'
SampleSubmission = pd.read_csv("data/SampleSubmission.csv")
sub = pd.merge(SampleSubmission.drop(columns='Active'), sub, on='User_ID_Next_month_Activity', how='left').fillna(0)
sub.to_csv("stacked_submission.csv", index=False)
